## .DOCX to .TXT file converter

#### docx package may need to be installed
#### To install enter command "pip install docx"
#### [Documentation can be found here](https://python-docx.readthedocs.io/en/latest/index.html#)

In [204]:
# !pip install docx
import docx

#### Global Variables to aid with filenames

In [205]:
path_to_data = '../data/Lexis Cases/'
file_prefix = 'P'
file_suffix = '.DOCX'
file_identifiers = range(1, 86) # Range from 1 to 85

out_path = '../data/Lexis Cases txt/'
out_file_suffix = '.txt'

#### Text Extraction

The library `docx` doesn't have simple support for going over each element in the document sequentially. I found a solution online that finds the children of each document and constructs new `Paragraph` and `Table` elements for the user. Code and link to the source can be found below.

In [206]:
# This code is sourced from: https://github.com/python-openxml/python-docx/issues/40#issuecomment-90710401
# with modification made from comment: 

# Currently, there is no built-in method of iterating over all items sequentially
# We must manually iterate over children and call the constructors of Paragraph, Table
# Code below summary:
#      1. Grab body of document
#      2. Check if the child in "document.element.body.iterchildren()" is CT_P, or CT_Tbl
#      3. Call the Paragraph or Table constructor with the child and its parent (the document)
#      4. Yield the paragraph/table for user to manipulate

from docx.document import Document
from docx.oxml.table import CT_Tbl
from docx.oxml.text.paragraph import CT_P
from docx.table import _Cell, Table
from docx.text.paragraph import Paragraph


def iter_block_items(parent):
    """
    Yield each paragraph and table child within *parent*, in document order.
    Each returned value is an instance of either Table or Paragraph. *parent*
    would most commonly be a reference to a main Document object, but
    also works for a _Cell object, which itself can contain paragraphs and tables.
    """
    if isinstance(parent, Document):
        parent_elm = parent.element.body
    elif isinstance(parent, _Cell):
        parent_elm = parent._tc
    else:
        raise ValueError("something's not right")

    for child in parent_elm.iterchildren():
        if isinstance(child, CT_P):
            yield Paragraph(child, parent)
        elif isinstance(child, CT_Tbl):
            yield Table(child, parent)

The code below streamlines opening each `.DOCX` file, reading the information, and writing it out to a `.txt` format. For tables our strategy is to concatenate each column for every row and treat it as a single sentence.

To get the case title we need to access the documents `header` via the documents `sections` and pull out text from row 1, cell 0. I believe the "page X of y" is store in row 0, cell 0 but this information isn't needed.

In [ ]:
for file_number in file_identifiers:
    print('Processing ' + path_to_data + file_prefix + str(file_number) + file_suffix)
    
    # Open the document
    # (Note: docx.Document is different from docx.document.Document)
    document = docx.Document(path_to_data + file_prefix + str(file_number) + file_suffix)
    file_text = ''

    start_of_case = True
    doc_section = 0
    
    # Iterate over each item in the body
    for item in iter_block_items(document):

        # If at start of case, store the header (case name)
        # as the first entry. Must access 'document.sections[].header' to do so.
        if start_of_case:
            header = document.sections[doc_section].header
            case_title = header.tables[0].cell(1, 0).text
            file_text += case_title.strip() + '\n'
            doc_section += 2
            start_of_case = False
        
        # If item is a PARAGRAPH
        if isinstance(item, Paragraph):
            # Skip over any empty lines
            if len(item.text.strip()) > 0:
                file_text += item.text.strip() + '\n'

            # Check if we are at the last line of a case
            # to make sure we insert the header of the following case
            if item.text.strip() == 'End of Document':
                start_of_case = True 
          
        # If item is a TABLE
        elif isinstance(item, Table): 
            # We group everything across a row as one "sentence" in our .txt file
            # Currently have no better ideas. This seems to be the most common format of tables in our .docx files
            for row in item.rows:
                text = [cell.text.strip() for cell in row.cells]
                file_text += ' '.join(text) + '\n'
        
            
    # Save as .txt
    with open(out_path + file_prefix + str(file_number) + out_file_suffix, 'w') as out_file:
        out_file.write(file_text)
        

Processing ../data/Lexis Cases/P1.DOCX
Processing ../data/Lexis Cases/P2.DOCX
Processing ../data/Lexis Cases/P3.DOCX
Processing ../data/Lexis Cases/P4.DOCX
Processing ../data/Lexis Cases/P5.DOCX
Processing ../data/Lexis Cases/P6.DOCX
Processing ../data/Lexis Cases/P7.DOCX
Processing ../data/Lexis Cases/P8.DOCX
Processing ../data/Lexis Cases/P9.DOCX
Processing ../data/Lexis Cases/P10.DOCX
Processing ../data/Lexis Cases/P11.DOCX
Processing ../data/Lexis Cases/P12.DOCX
Processing ../data/Lexis Cases/P13.DOCX
Processing ../data/Lexis Cases/P14.DOCX
Processing ../data/Lexis Cases/P15.DOCX
Processing ../data/Lexis Cases/P16.DOCX
Processing ../data/Lexis Cases/P17.DOCX
Processing ../data/Lexis Cases/P18.DOCX
Processing ../data/Lexis Cases/P19.DOCX
Processing ../data/Lexis Cases/P20.DOCX
Processing ../data/Lexis Cases/P21.DOCX
Processing ../data/Lexis Cases/P22.DOCX
Processing ../data/Lexis Cases/P23.DOCX
Processing ../data/Lexis Cases/P24.DOCX
Processing ../data/Lexis Cases/P25.DOCX
Processin